## Get a filtration from Gudhi

In [1]:
import gudhi as gd
import tadasets
sphere = tadasets.dsphere(n=30, d=2, r=1, noise=0, seed=42)
alpha = gd.AlphaComplex(points=sphere)
st = alpha.create_simplex_tree()
filtration = list(st.get_filtration())

## Convert it to boundary map data

In [2]:
from src.filtrations import simplices_by_dimension, compute_boundary_matrices, filtration_hash_map

unique_filtration_values = sorted(list(set([f for (_, f) in filtration])))
simplices_by_dim, simplices_by_dim_only_filt = simplices_by_dimension(filtration)
boundary_matrices = compute_boundary_matrices(simplices_by_dim)
boundary_maps_index_dict = filtration_hash_map(filtration, simplices_by_dim_only_filt)

## Compute homology using persistent laplacians
For now, compute only first homology.
Lanczos library panics on diagonal matrices it seems.

In [3]:
import persistent_laplacians
result = persistent_laplacians.process_tda(
    boundary_matrices,
    boundary_maps_index_dict
)

## Barcodes 

In [4]:
from src.barcodes import barcodes
barcodes = barcodes(result, unique_filtration_values)
print(barcodes)
barcodes = [
    (q, (unique_filtration_values[i], unique_filtration_values[j]))
    for q in barcodes.keys()
    for (i, j) in barcodes[q].keys()
]
sorted(barcodes)

{2: {(131, 133): 1, (133, 135): 1, (131, 137): 1, (126, 138): 1, (126, 129): 1, (120, 139): 1, (106, 140): 1}, 1: {(99, 100): 1, (59, 60): 1, (50, 52): 1, (74, 75): 1, (36, 37): 1, (9, 14): 1, (81, 88): 1, (68, 79): 1, (17, 18): 1, (32, 34): 1, (12, 19): 1, (85, 86): 1, (69, 70): 1, (54, 97): 1, (64, 101): 1, (40, 45): 1, (41, 51): 1, (76, 77): 1, (80, 89): 1, (55, 61): 1, (62, 67): 1}, 3: {}}


[(1, (0.02546175064832349, 0.027562930776185896)),
 (1, (0.026800551472400735, 0.03112329831866171)),
 (1, (0.02906069519472558, 0.0292751263572475)),
 (1, (0.07867295640614468, 0.08458221447589057)),
 (1, (0.08694565715396549, 0.0869456580426308)),
 (1, (0.09115194518810042, 0.10539997067827911)),
 (1, (0.09571882287087163, 0.11643740730637718)),
 (1, (0.113321226926027, 0.11706756724000403)),
 (1, (0.12695150583184348, 0.3812157210058862)),
 (1, (0.13470690124391815, 0.1552750672070925)),
 (1, (0.1516271641567509, 0.1516427338878758)),
 (1, (0.15865363644614866, 0.18101916166066753)),
 (1, (0.17163697847394072, 0.4619707011897518)),
 (1, (0.19174794328725328, 0.25297922765318015)),
 (1, (0.1923996547957571, 0.1950190126995972)),
 (1, (0.21427641695380947, 0.21429864552772004)),
 (1, (0.22000470980115436, 0.2220132176376405)),
 (1, (0.2530665239584188, 0.3341025442401989)),
 (1, (0.25717912745174704, 0.3312652681257092)),
 (1, (0.29956537923025706, 0.30270646472564455)),
 (1, (0.41774

## Verify with Gudhi

In [6]:
gudhi_persistence = [(q, (i, j)) for (q, (i,j)) in st.persistence() if q > 0]
sorted(gudhi_persistence) == sorted(barcodes)

True